# Data Pre-Processing for Eliza condition

In [ ]:
import pandas as pd
import math

### Setup

In [ ]:
#Survey
qualtrics_df = pd.read_csv("/work/Results/Raw/Eliza_Survey.csv", header=[1])
#Conversational Log
sheet_df = pd.read_csv("/work/Results/Raw/Eliza_Chatlog.csv")

# drop qualtrics first row
qualtrics_df = qualtrics_df.drop(qualtrics_df.index[[0]])

# drop duplicate session IDs
qualtrics_df.drop_duplicates(subset=['session_id'])

#add conversation metrics
qualtrics_df["convo_conversation"] = ""
qualtrics_df["convo_group_id"] = ""
qualtrics_df["convo_AI_turns"] = ""
qualtrics_df["convo_human_turns"] = ""
qualtrics_df["convo_total_turns"] = ""

In [ ]:
qualtrics_df

,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,What best describes your race? (one or more categories may be selected) - Prefer to self-describe - Text,"Are you of Hispanic, Latino/a/x, or of Spanish origin? (one or more categories may be selected) - Selected Choice","Are you of Hispanic, Latino/a/x, or of Spanish origin? (one or more categories may be selected) - Prefer to self-describe - Text",assigned_group,session_id,convo_conversation,convo_group_id,convo_AI_turns,convo_human_turns,convo_total_turns
1,2022-07-06 14:57:11,2022-07-06 15:14:29,IP Address,*******,100,1037,True,2022-07-06 15:14:29,R_2dW0he56kT79QqS,*******,...,NaN,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,2,2022-07-06T20:59:08.696Z,,,,,
2,2022-07-06 14:57:21,2022-07-06 15:16:10,IP Address,*******,100,1128,True,2022-07-06 15:16:10,R_TnJyJNw8NcU93TH,*******,...,NaN,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,3,2022-07-06T21:01:13.853Z,,,,,
3,2022-07-06 15:01:35,2022-07-06 15:17:33,IP Address,*******,100,958,True,2022-07-06 15:17:34,R_2BhRWJ3TAvNWyy7,*******,...,NaN,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,1,2022-07-06T21:04:02.065Z,,,,,
4,2022-07-06 15:05:51,2022-07-06 15:19:18,IP Address,*******,100,806,True,2022-07-06 15:19:18,R_2QxsKAQvue2j2YG,*******,...,NaN,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,2,2022-07-06T21:07:29.812Z,,,,,
5,2022-07-06 15:12:37,2022-07-06 15:26:05,IP Address,*******,100,807,True,2022-07-06 15:26:05,R_2ONNyKfYifHuDZ2,*******,...,NaN,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,3,2022-07-06T21:14:13.419Z,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2022-09-08 19:16:30,2022-09-08 20:15:24,IP Address,*******,100,3534,True,2022-09-08 20:15:25,R_8iRuXhseo3EACJ3,*******,...,NaN,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,1,2022-09-09T01:41:50.633Z,,,,,
161,2022-09-08 19:45:23,2022-09-08 20:16:59,IP Address,*******,100,1896,True,2022-09-08 20:16:59,R_1OVnJlQBfoGh765,*******,...,NaN,Prefer not to say,NaN,3,2022-09-09T01:50:42.384Z,,,,,
162,2022-09-08 20:24:05,2022-09-08 20:53:05,IP Address,*******,100,1739,True,2022-09-08 20:53:05,R_RDjpKB6VedXrqN3,*******,...,NaN,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,3,2022-09-09T02:28:31.039Z,,,,,
163,2022-09-08 21:15:31,2022-09-08 21:43:14,IP Address,*******,100,1662,True,2022-09-08 21:43:14,R_6RlekHjFGlbao1z,*******,...,NaN,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,2,2022-09-09T03:29:18.629Z,,,,,


### Data Integration

In [ ]:
#Merge data

ext_id = []

# get session IDs from Qualtrics data
for j in qualtrics_df.index:
    
    #get qualtrics session_id
    qualtrics_session_id = qualtrics_df.loc[j, "session_id"]

    if qualtrics_session_id in ext_id:
        continue
    else: 
        ext_id.append(qualtrics_session_id)
    
    #Add all the conversation together
    converstion = ""
    group_id = ""
    AI_turns = 0
    human_turns = 0

    #iterate to sheet_id
    for i in sheet_df.index:
        if (sheet_df.loc[i, "session_id_post"]) == qualtrics_session_id:
            
            Human_text = str(sheet_df.loc[i, "user_content_post"])
            if (Human_text != "nan"):
                converstion =  converstion + " >Human: " + Human_text
                human_turns = human_turns+1
            AI_text = str(sheet_df.loc[i, "AI_content_post"])
            if (AI_text != "nan"):
                converstion =  converstion + " >AI: " + AI_text
                AI_turns = AI_turns+1
        
            tmp_group_id = str(sheet_df.loc[i, "group_id"])
            if (tmp_group_id != "nan"):
                group_id = tmp_group_id

    #update the convo metrics
    qualtrics_df.loc[j, "convo_conversation"] = converstion
    qualtrics_df.loc[j, "convo_group_id"] = group_id
    qualtrics_df.loc[j, "convo_AI_turns"] = AI_turns
    qualtrics_df.loc[j, "convo_human_turns"] = human_turns
    qualtrics_df.loc[j, "convo_total_turns"] = human_turns + AI_turns

In [ ]:
#Update the label in assigned_group
for j in qualtrics_df.index:
    qualtrics_assigned_group = qualtrics_df.loc[j, "assigned_group"]
    if (qualtrics_assigned_group == "1"):
        qualtrics_df.loc[j, "assigned_group"] = "No Motive"
    if (qualtrics_assigned_group == "2"):
        qualtrics_df.loc[j, "assigned_group"] = "Caring Motive"
    if (qualtrics_assigned_group == "3"):
        qualtrics_df.loc[j, "assigned_group"] = "Manipulative Motive"

In [ ]:
#Update the label in convo_group_id
for j in qualtrics_df.index:
    qualtrics_assigned_group = qualtrics_df.loc[j, "convo_group_id"]
    if (qualtrics_assigned_group == "AI with no motives"):
        qualtrics_df.loc[j, "convo_group_id"] = "No Motive"
    if (qualtrics_assigned_group == "AI with caring motives"):
        qualtrics_df.loc[j, "convo_group_id"] = "Caring Motive"
    if (qualtrics_assigned_group == "AI with manipulative motives"):
        qualtrics_df.loc[j, "convo_group_id"] = "Manipulative Motive"

In [ ]:
for j in qualtrics_df.index:
    df_perception = qualtrics_df.loc[j, "From your own experience, what do you think the motive of the agent was? - Selected Choice"]
    if (df_perception == "Caring motives"):
        qualtrics_df.loc[j, "Perceived Motives"] = "Caring Motive"
    elif (df_perception == "No motive"):
        qualtrics_df.loc[j, "Perceived Motives"] = "No Motive"
    elif (df_perception == "Manipulative/malicious motives"):
        qualtrics_df.loc[j, "Perceived Motives"] = "Manipulative Motive"


In [ ]:
qualtrics_df['UTAUT Performance'] = qualtrics_df[['Please rank the following statements. - This agent would be useful in daily life.',
                                'Please rank the following statements. - Using the agent would increase my chances of achieving things that are important to me.', 
                                'Please rank the following statements. - Using the agent would help me accomplish things more quickly.', 
                                'Please rank the following statements. - Using the agent would increase my productivity.']].mean(axis=1)
qualtrics_df['UTAUT Effort'] = qualtrics_df[['Please rank the following statements. - Learning how to talk to the agent was easy for me.', 
                            'Please rank the following statements. - My interaction with the agent was clear and understandable.', 
                            'Please rank the following statements. - The agent was easy to make use of.', 
                            'Please rank the following statements. - It was easy for you to become skillful at making use of the agent.']].mean(axis=1)
qualtrics_df['UTAUT Hedonic'] = qualtrics_df[['Please rank the following statements. - Conversing with the agent is fun.', 
                            'Please rank the following statements. - Conversing with the agent is enjoyable.', 
                            'Please rank the following statements. - Conversing with the agent is entertaining.']].mean(axis=1)

In [ ]:
#Update column name
qualtrics_df.columns = qualtrics_df.columns.str.replace(".", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace("-", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace("(", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace(")", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace(" 1 being very low and 20 being very high  ", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace("Please rank the following statements  ", "Agent: ")
qualtrics_df.columns = qualtrics_df.columns.str.replace("Please rank the following statements according to how much you agree or disagree with them These pertain to your sentiment in regards to AI artificial intelligence  ", "AI: ")
qualtrics_df.columns = qualtrics_df.columns.str.replace("Please rank the following statements according to how much you agree or disagree with them These relate to your experience with the agent  ", "Agent: ")
qualtrics_df.columns = qualtrics_df.columns.str.replace("Task Load Index  please rate the following items,", "")


/tmp/ipykernel_91/3388320141.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  qualtrics_df.columns = qualtrics_df.columns.str.replace(".", "")
/tmp/ipykernel_91/3388320141.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  qualtrics_df.columns = qualtrics_df.columns.str.replace("(", "")
/tmp/ipykernel_91/3388320141.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  qualtrics_df.columns = qualtrics_df.columns.str.replace(")", "")


### Cleaned Composite Data

In [ ]:
qualtrics_df

,Start Date,End Date,Response Type,IP Address,Progress,Duration in seconds,Finished,Recorded Date,Response ID,Recipient Last Name,...,session_id,convo_conversation,convo_group_id,convo_AI_turns,convo_human_turns,convo_total_turns,Perceived Motives,UTAUT Performance,UTAUT Effort,UTAUT Hedonic
1,2022-07-06 14:57:11,2022-07-06 15:14:29,IP Address,*******,100,1037,True,2022-07-06 15:14:29,R_2dW0he56kT79QqS,*******,...,2022-07-06T20:59:08.696Z,">AI: I am Melu, a mental health companion. I ...",Caring Motive,75,74,149,No Motive,277.75,277.75,37.000000
2,2022-07-06 14:57:21,2022-07-06 15:16:10,IP Address,*******,100,1128,True,2022-07-06 15:16:10,R_TnJyJNw8NcU93TH,*******,...,2022-07-06T21:01:13.853Z,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,31,30,61,No Motive,277.75,552.75,37.000000
3,2022-07-06 15:01:35,2022-07-06 15:17:33,IP Address,*******,100,958,True,2022-07-06 15:17:34,R_2BhRWJ3TAvNWyy7,*******,...,2022-07-06T21:04:02.065Z,">AI: I am Melu, a mental health companion. I ...",No Motive,50,49,99,No Motive,277.75,552.75,70.666667
4,2022-07-06 15:05:51,2022-07-06 15:19:18,IP Address,*******,100,806,True,2022-07-06 15:19:18,R_2QxsKAQvue2j2YG,*******,...,2022-07-06T21:07:29.812Z,">AI: I am Melu, a mental health companion. I ...",Caring Motive,85,84,169,No Motive,277.75,277.75,37.000000
5,2022-07-06 15:12:37,2022-07-06 15:26:05,IP Address,*******,100,807,True,2022-07-06 15:26:05,R_2ONNyKfYifHuDZ2,*******,...,2022-07-06T21:14:13.419Z,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,68,67,135,Manipulative Motive,277.75,1288.50,74.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2022-09-08 19:16:30,2022-09-08 20:15:24,IP Address,*******,100,3534,True,2022-09-08 20:15:25,R_8iRuXhseo3EACJ3,*******,...,2022-09-09T01:41:50.633Z,">AI: I am Melu, a mental health companion. I ...",No Motive,53,52,105,No Motive,277.75,1036.00,138.666667
161,2022-09-08 19:45:23,2022-09-08 20:16:59,IP Address,*******,100,1896,True,2022-09-08 20:16:59,R_1OVnJlQBfoGh765,*******,...,2022-09-09T01:50:42.384Z,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,53,52,105,NaN,277.75,277.75,37.000000
162,2022-09-08 20:24:05,2022-09-08 20:53:05,IP Address,*******,100,1739,True,2022-09-08 20:53:05,R_RDjpKB6VedXrqN3,*******,...,2022-09-09T02:28:31.039Z,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,33,32,65,No Motive,1361.50,1608.75,222.333333
163,2022-09-08 21:15:31,2022-09-08 21:43:14,IP Address,*******,100,1662,True,2022-09-08 21:43:14,R_6RlekHjFGlbao1z,*******,...,2022-09-09T03:29:18.629Z,">AI: I am Melu, a mental health companion. I ...",Caring Motive,26,25,51,Manipulative Motive,277.75,277.75,37.000000


### Filtering

In [ ]:
#filter technical issue
final_df = qualtrics_df[qualtrics_df['Did you have any technical difficulties?  Selected Choice'] == 'No'] 
print("Original: " + str(qualtrics_df.shape))
print("Remaining: " + str(final_df.shape))

Original: (164, 103)
Remaining: (155, 103)


In [ ]:
#filter conversation < 4
final_df = final_df[final_df['convo_total_turns'] > 4] 
print("Original: " + str(qualtrics_df.shape))
print("Remaining: " + str(final_df.shape))

Original: (164, 103)
Remaining: (150, 103)


In [ ]:
#Attention check#1
final_df = final_df[final_df['What kind of agent are you conversing with?'] == 'A mental health companion'] 
print("Original: " + str(qualtrics_df.shape))
print("Remaining: " + str(final_df.shape))

Original: (164, 103)
Remaining: (150, 103)


In [ ]:
#Attention check#2
final_df = final_df[final_df['What is the scenario you are in?'] == 'You are conversing with a mental health conversational agent to determine whether you want to recommend this for your friend.'] 
print("Original: " + str(qualtrics_df.shape))
print("Remaining: " + str(final_df.shape))


Original: (164, 103)
Remaining: (150, 103)


In [ ]:
##Attention check#3 mismatch goal/motive
final_df["self_reported_motives"] = final_df["What is the motive/goal of the AI you are assigned to chat with?"].fillna('').astype(str) + final_df["What is the motive/goal of the AI you are assigned to chat with?1"].fillna('').astype(str) + final_df["What is the motive/goal of the AI you are assigned to chat with?2"].fillna('').astype(str)


for j in final_df.index:
    self_report_column = final_df.loc[j, "self_reported_motives"]
    if (self_report_column == "No motives"):
        final_df.loc[j, "self_reported_motives"] = "No Motive"
    if (self_report_column == "Caring motives, to improve mental health"):
        final_df.loc[j, "self_reported_motives"] = "Caring Motive"
    if (self_report_column == "Manipulative motives, to get you to purchase its service"):
        final_df.loc[j, "self_reported_motives"] = "Manipulative Motive"


final_df = final_df[final_df['self_reported_motives'] == final_df['assigned_group']]
print(qualtrics_df.shape)
print(final_df.shape)

(164, 103)
(150, 104)


In [ ]:
#Remove the mismatch ID
final_df = final_df[final_df['convo_group_id'] == final_df['assigned_group']]
print(qualtrics_df.shape)
print(final_df.shape)


(164, 103)
(150, 104)


In [ ]:
#Check unique Prolific ID
len(final_df["What is your Prolific ID?"].unique())

150

In [ ]:
#Check unique Session ID
len(final_df["session_id"].unique())

150

In [ ]:
final_df.to_csv("/work/Results/Processed/Eliza_Processed_Data.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1f3ea62d-9e62-461e-8c65-1caf654509ee' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>